In [6]:
import sqlite3
class Kitap:
    def __init__(self, baslik, yazar, yayin_evi, yil):
        self.baslik = baslik
        self.yazar = yazar
        self.yayin_evi = yayin_evi
        self.yil = yil
    def bilgileri_goster(self):
        print(50 * "#")
        print(f"""
        Baslik:{self.baslik}
        Yazar:{self.yazar}
        Yayinevi:{self.yayin_evi}
        Yayin Yili:{self.yil}
        """)
class Uye:
    def __init__(self, uye_adi, uye_soyadi, uye_no):
        self.uye_adi = uye_adi
        self.uye_soyadi = uye_soyadi
        self.uye_no = uye_no
    def bilgileri_goster(self):
        print(50 * "#")
        print(f""" 
        Uye Adi:{self.uye_adi}
        Uye Soyadi:{self.uye_soyadi}
        Uye Numarasi:{self.uye_no}
        """)
class Kutuphane:
    def __init__(self):
        self.conn = sqlite3.connect("kutuphane.db")
        self.cursor = self.conn.cursor()
        self._create_tables()
    def _create_tables(self):
        self.cursor.execute('''
        create table if not exists Kitaplar(
        id integer primary key autoincrement,
        baslik varchar(300),
        yazar varchar(100),
        yayin_evi varchar(200),
        yil date)
        ''')
        self.cursor.execute('''
        create table if not exists Uyeler(
        id integer primary key autoincrement,
        uye_adi varchar(100),
        uye_soyadi varchar(100),
        uye_no varchar(11)
        )
        ''')
        self.cursor.execute('''
        create table if not exists AlinanKitaplar(
        kitap_id integer,
        uye_id integer,
        Foreign key(kitap_id) references Kitaplar(id)
        Foreign key(uye_id) references Uyeler(id)
        primary key(kitap_id, uye_id)
        )
        ''')
        self.conn.commit()
    def _get_kitap_id(self, baslik):
        self.cursor.execute('select id from Kitaplar where baslik = ?', (baslik,))
        result = self.cursor.fetchone()
        return result[0] if result else None
    def _get_uye_id(self, uye_no):
        self.cursor.execute('select id from Uyeler where uye_no = ?', (uye_no,))
        result = self.cursor.fetchone()
        return result[0] if result else None
    def kitap_ekle(self, kitap):
        self.cursor.execute('insert into Kitaplar(baslik, yazar, yayin_evi,yil) values(?,?,?,?)', 
                            (kitap.baslik,kitap.yazar,kitap.yayin_evi,kitap.yil))
        self.conn.commit()
        print(f"{kitap.baslik} basarili sekilde kutuphaneye eklendi.")
    def kitap_sil(self, baslik):
        kitap_id = self._get_kitap_id(baslik)
        if kitap_id:
            self.cursor.execute('delete from Kitaplar where id =?', (kitap_id,))
            self.conn.commit()
            print(f"{baslik} basarili sekilde kutuphaneden silindi.")
        else:
            print(f"{baslik} boyle bir kitap tespit edilemedi.")
    def uye_ekle(self, uye):
        self.cursor.execute('insert into Uyeler(uye_adi, uye_soyadi, uye_no) values(?,?,?)',
                            (uye.uye_adi, uye.uye_soyadi, uye.uye_no))
        self.conn.commit()
        print(f"{uye.uye_adi} {uye.uye_soyadi} basarili sekilde eklendi.")
    def uye_sil(self, uye_no):
        uye_id = self._get_uye_id(uye_no)
        if uye_id:
            self.cursor.execute('delete from Uyeler where id = ?', (uye_id,))
            print(f"{uye_no} basarili sekilde silindi.")
        else:
            print(f"{uye_no} numaraya ait uye bulunamadi.")
    def odunc_alma(self, baslik, uye_no):
        kitap_id = self._get_kitap_id(baslik)
        uye_id = self._get_uye_id(uye_no)
        if kitap_id and uye_id:
            self.cursor.execute('select * from AlinanKitaplar where kitap_id = ?', (kitap_id,))
            odunc_durum = self.cursor.fetchone()
            if odunc_durum:
                print(f"{baslik} ilgili kitap baska uyede, baska bir kitap seciniz.")
            else:
                self.cursor.execute('insert into AlinanKitaplar(kitap_id, uye_id) values(?,?)', (kitap_id, uye_id))
                self.conn.commit()
                print(f"{baslik} ilgili kitap {uye_no} numarali uye tarafindan alindi.")
        else:
            print("Hatali kitap veya uye numarasi girildi.")
    def odunc_verme(self, baslik):
        kitap_id = self._get_kitap_id(baslik)
        if kitap_id:
            self.cursor.execute('select * from AlinanKitaplar where kitap_id = ?',(kitap_id,))
            odunc_durumu = self.cursor.fetchone()
            if odunc_durumu:
                self.cursor.execute('delete from AlinanKitaplar where kitap_id = ? ', (kitap_id,))
                self.conn.commit()
                print(f"{baslik} ilgili kitap geri verildi.")
            else:
                print(f"{baslik} kitap odunc alinmamistir.")
        else:
            print("Yanlis bir kitap girildi.")
    def kitaplari_goster(self):
        self.cursor.execute('select * from Kitaplar')
        kitaplar = self.cursor.fetchall()
        if not kitaplar:
            print("Kutuphaneye eklenmis kitap bulunmamaktadir.")
        else:
            print("Kitaplar: ")
            for kitap in kitaplar:
                kitap_obj = Kitap(*kitap[1:])
                kitap_obj.bilgileri_goster()
    def uyeleri_goster(self):
        self.cursor.execute('select * from Uyeler')
        uyeler = self.cursor.fetchall()
        if not uyeler:
            print("Herhangi bir uye bulunmamaktadir.")
        else:
            print("Uyeler: ")
            for uye in uyeler:
                uye_obj = Uye(*uye[1:])
                uye_obj.bilgileri_goster()
    def close_connection(self):
        self.conn.close()
kutuphane = Kutuphane()
while True:
    print(30 * "*", "Kutuphane Yonetım Sistemi", 30 * "*")
    print("""
    1.Kitap Ekle
    2.Kitap Sil
    3.Uye Ekle
    4.Uye Sil
    5.Kitap Alma
    6.Kitap Teslimi
    7.Kitaplari Goster
    8.Uyeleri Goster
    9.Cikis ('Q-q')
    """)
    islem = input("İlgili islemin numarasisini girin: ")
    if islem == "1":
        baslik = input("Baslik: ")
        yazar = input("Yazar: ")
        yayin_evi = input("Yayinevi: ")
        yil = input("Yayin yilini girin: ")
        kitap = Kitap(baslik, yazar, yayin_evi, yil)
        kutuphane.kitap_ekle(kitap)
    elif islem == "2":
        baslik = input("Silinecek kitabin basligini girin: ")
        kutuphane.kitap_sil(baslik)
    elif islem == "3":
        uye_adi = input("Uye adi: ")
        uye_soyadi = input("Uye soyadin: ")
        uye_no = input("Uye numarasi: ")
        uye = Uye(uye_adi, uye_soyadi, uye_no)
        kutuphane.uye_ekle(uye)
    elif islem == "4":
        uye_no = input("Silmek istediginiz uye no: ")
        kutuphane.uye_sil(uye_no)
    elif islem == "5":
        baslik = input("Alacaginiz kitabin adini girin: ")
        uye_no = input("Uye no girin: ")
        kutuphane.odunc_alma(baslik, uye_no)
    elif islem == "6":
        baslilk = input("Teslim edilecek kitap basligini girin: ")
        kutuphane.odunc_verme(baslik)
    elif islem == "7":
        kutuphane.kitaplari_goster()
    elif islem == "8":
        kutuphane.uyeleri_goster()
    elif islem == "9" or islem == "q" or islem == "Q":
        print("Uygulamadan cikis yapildi.")
        break
    else:
        print("Yanlis bir islem numarasi girildi (1-9).")
kutuphane.close_connection()

****************************** Kutuphane Yonetım Sistemi ******************************

    1.Kitap Ekle
    2.Kitap Sil
    3.Uye Ekle
    4.Uye Sil
    5.Kitap Alma
    6.Kitap Teslimi
    7.Kitaplari Goster
    8.Uyeleri Goster
    9.Cikis ('Q-q')
    
İlgili islemin numarasisini girin: 2
Silinecek kitabin basligini girin: Dijital Kale
Dijital Kale basarili sekilde kutuphaneden silindi.
****************************** Kutuphane Yonetım Sistemi ******************************

    1.Kitap Ekle
    2.Kitap Sil
    3.Uye Ekle
    4.Uye Sil
    5.Kitap Alma
    6.Kitap Teslimi
    7.Kitaplari Goster
    8.Uyeleri Goster
    9.Cikis ('Q-q')
    
İlgili islemin numarasisini girin: 4
Silmek istediginiz uye no: 1
1 basarili sekilde silindi.
****************************** Kutuphane Yonetım Sistemi ******************************

    1.Kitap Ekle
    2.Kitap Sil
    3.Uye Ekle
    4.Uye Sil
    5.Kitap Alma
    6.Kitap Teslimi
    7.Kitaplari Goster
    8.Uyeleri Goster
    9.Cikis ('Q-q')
   